In [ ]:
import os
import json
from PIL import Image

# Define folders
image_folder = "images" # Image folder
metadata_folder = "metadata" # labelme output
output_folder = "labels" # yolo format .txt files

# Create output folder if not exists
os.makedirs(output_folder, exist_ok=True)

# Define class mapping (add all your classes here)
class_mapping = {"Pothole": 0}

# Iterate over all JSON files
for json_file in os.listdir(metadata_folder):
    if not json_file.endswith(".json"):
        continue

    # Load JSON annotation
    with open(os.path.join(metadata_folder, json_file)) as f:
        data = json.load(f)

    # Derive corresponding image path
    image_path = os.path.join(image_folder, data["imagePath"])
    image = Image.open(image_path)
    img_width, img_height = image.size

    # Create output .txt file
    txt_filename = os.path.splitext(json_file)[0] + ".txt"
    txt_path = os.path.join(output_folder, txt_filename)

    with open(txt_path, "w") as out:
        for shape in data["shapes"]:
            print(shape)
            label = shape["label"]
            points = shape["points"]

            # Extract bounding box (xmin, ymin), (xmax, ymax)
            x1, y1 = points[0]
            x2, y2 = points[1]
            x_min, x_max = sorted([x1, x2])
            y_min, y_max = sorted([y1, y2])

            # Compute YOLO format
            x_center = (x_min + x_max) / 2.0 / img_width
            y_center = (y_min + y_max) / 2.0 / img_height
            width = (x_max - x_min) / img_width
            height = (y_max - y_min) / img_height

            class_id = class_mapping[label]
            out.write(
                f"{class_id} {x_center:.6f} {y_center:.6f} {width:.6f} {height:.6f}\n"
            )

print("✅ Conversion complete. YOLO .txt files are saved in the 'labels' folder.")

In [ ]:
import cv2
import os

# === Configuration ===
image_path = "/Users/manojkl/Documents/Project/Pothole_detection/Experiments/dataset/data/images/78.jpg"  # path to your image
label_path = "/Users/manojkl/Documents/Project/Pothole_detection/Experiments/dataset/data/labels/78.txt"  # path to corresponding YOLO .txt file
class_names = ["Pothole"]  # list of class names (index 0 means "ear")

# === Load image ===
image = cv2.imread(image_path)
img_height, img_width = image.shape[:2]

# === Read YOLO annotations and draw boxes ===
with open(label_path, "r") as f:
    for line in f:
        parts = line.strip().split()
        class_id = int(parts[0])
        x_center = float(parts[1]) * img_width
        y_center = float(parts[2]) * img_height
        width = float(parts[3]) * img_width
        height = float(parts[4]) * img_height

        # Convert to top-left corner
        x1 = int(x_center - width / 2)
        y1 = int(y_center - height / 2)
        x2 = int(x_center + width / 2)
        y2 = int(y_center + height / 2)

        # Draw rectangle and label
        cv2.rectangle(image, (x1, y1), (x2, y2), (0, 255, 0), 2)
        label = class_names[class_id]
        cv2.putText(
            image, label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2
        )

# === Show the image ===
cv2.imshow("Annotated Image", image)
cv2.waitKey(0)
cv2.destroyAllWindows()

# Optionally save the result
cv2.imwrite("output_with_boxes.png", image)

In [ ]:
import os
import random
import shutil
from pathlib import Path

# Set the random seed for reproducibility
random.seed(42)

# Input directories
images_dir = Path(
    "/Users/manojkl/Documents/Project/Pothole_detection/Experiments/dataset/data/images"
)
labels_dir = Path(
    "/Users/manojkl/Documents/Project/Pothole_detection/Experiments/dataset/data/labels"
)

# Output base directories
output_base_images = Path(
    "/Users/manojkl/Documents/Project/Pothole_detection/Experiments/zip3/images"
)
output_base_labels = Path(
    "/Users/manojkl/Documents/Project/Pothole_detection/Experiments/zip3/labels"
)

# Ratios
train_ratio = 0.7
val_ratio = 0.2
test_ratio = 0.1

# Create output folders
splits = ["train", "val", "test"]
for split in splits:
    (output_base_images / split).mkdir(parents=True, exist_ok=True)
    (output_base_labels / split).mkdir(parents=True, exist_ok=True)

# Get all image files (assuming .jpg or .png and corresponding .txt labels)
image_files = [f for f in images_dir.glob("*.*") if f.suffix in [".jpg", ".png"]]
image_files = [
    f for f in image_files if (labels_dir / f.with_suffix(".txt").name).exists()
]

# Shuffle dataset
random.shuffle(image_files)

# Split
total = len(image_files)
train_end = int(train_ratio * total)
val_end = train_end + int(val_ratio * total)

train_files = image_files[:train_end]
val_files = image_files[train_end:val_end]
test_files = image_files[val_end:]

# Helper function to copy image-label pairs
def copy_files(file_list, split):
    for image_path in file_list:
        label_path = labels_dir / image_path.with_suffix(".txt").name
        if label_path.exists():
            shutil.copy(image_path, output_base_images / split / image_path.name)
            shutil.copy(label_path, output_base_labels / split / label_path.name)


# Copy files
copy_files(train_files, "train")
copy_files(val_files, "val")
copy_files(test_files, "test")

print(
    f"Dataset split complete: {len(train_files)} train, {len(val_files)} val, {len(test_files)} test"
)